In [3]:
import dash
from dash import dcc, html
import pandas as pd


In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the SpaceX launch data into a DataFrame (assuming the CSV file is in the same directory as your script)
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

# Create the Dash app
app = dash.Dash(__name__)

# Create a list of launch sites from the dataset
launch_sites = spacex_df['Launch Site'].unique().tolist()

# Generate options for each launch site
site_options = [{'label': site, 'value': site} for site in launch_sites]

# Define the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),  # Title
    
    # Dropdown for launch sites
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            *site_options  # Unpack the site options
        ],
        value='ALL',  # Default value is set to 'ALL'
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    
    # Range slider for payload
    dcc.RangeSlider(
        id='payload-slider',
        min=0,  # Slider starting point (0 Kg)
        max=10000,  # Slider ending point (10,000 Kg)
        step=1000,  # Slider interval (1000 Kg)
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()],  # Initial payload range
        marks={0: '0', 10000: '10000'}  # Marks for slider
    ),

    # Pie chart to display success counts
    dcc.Graph(id='success-pie-chart'),

    # Scatter plot to display payload vs success
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback function to update the success pie chart and scatter plot based on the selected site and payload range
@app.callback(
    [Output(component_id='success-pie-chart', component_property='figure'),
     Output(component_id='success-payload-scatter-chart', component_property='figure')],
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_figures(selected_site, selected_payload_range):
    if selected_site == 'ALL':
        # Create a pie chart for success rate of each site
        pie_fig = px.pie(
            spacex_df,
            names='Launch Site',
            title='Success Rate for Each Launch Site'
        )
        # Filter data based on payload range for all sites
        filtered_data = spacex_df[
            (spacex_df['Payload Mass (kg)'] >= selected_payload_range[0]) &
            (spacex_df['Payload Mass (kg)'] <= selected_payload_range[1])
        ]
        # Create a scatter plot for all sites
        scatter_fig = px.scatter(
            filtered_data,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Payload vs Success for All Sites'
        )
    else:
        # Filter data for the selected site and payload range
        selected_site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        filtered_data = selected_site_data[
            (selected_site_data['Payload Mass (kg)'] >= selected_payload_range[0]) &
            (selected_site_data['Payload Mass (kg)'] <= selected_payload_range[1])
        ]
        # Create a pie chart for the selected site
        pie_fig = px.pie(
            filtered_data,
            names='class',
            title=f'Launch Success vs Failures for {selected_site}'
        )
        # Create a scatter plot for the selected site and payload range
        scatter_fig = px.scatter(
            filtered_data,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Payload vs Success for {selected_site}'
        )
    
    return pie_fig, scatter_fig

if __name__ == '__main__':
    app.run_server(debug=True)
